In [ ]:
import os
os.chdir('..')

from util import CONFIG
CONFIG.use_mpl(True)

## Section 7.1

Listing 7.1

In [ ]:
from sim_circuit import QuantumRegister, QuantumCircuit
from math import pi

def encode_frequency(n, v):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n):
        qc.h(q[j])

    for j in range(n):
        qc.p(2 * pi / 2 ** (n - j) * v, q[j]) # <1>

    qc.report('geometric_sequence') # <2>

    qc.iqft(range(n))

    qc.report('iqft')

    return qc

Listing 7.2

In [ ]:
from util import cis, prod

def complex_sincd(n, v):
    N = 2 ** n
    return [prod(
        cos((v - k) * pi / 2 ** (j + 1)) * cis((v - k) * pi / 2 ** (j + 1))
        for j in range(n)) for k in range(2 ** n)]

In [ ]:
n = 3

qc1 = encode_frequency(n, 3)

qc2 = encode_frequency(n, 3.8)

In [ ]:
result = qc1.measure(shots = 10)

In [ ]:
result['counts']

In [ ]:
from util import plot_bars

for outcome in range(2**n):
    result['counts'][outcome] = result['counts'].get(outcome, 0)
    
plot_bars(dict(sorted(result['counts'].items())), '', 'Outcomes', 'Frequency', color='grey')

In [ ]:
result = qc2.measure(shots = 10)

In [ ]:
result['counts']

In [ ]:
for outcome in range(2**n):
    result['counts'][outcome] = result['counts'].get(outcome, 0)
    
plot_bars(dict(sorted(result['counts'].items())), '', 'Outcomes', 'Frequency', color='grey')

#### Section 7.1.1

In [ ]:
n = 3
v = 4.76
qc = encode_frequency(n, v)

In [ ]:
from util import list_to_dict

state = qc.run()
probs = [abs(a)**2 for a in state]
plot_bars(list_to_dict(probs, False), f'v = {v}', 'Outcomes', 'Probabilities')

In [ ]:
result = qc.measure(shots = 100)

result['counts']

In [ ]:
n = 5
v = 19.05
qc = encode_frequency(n, v)

In [ ]:
state = qc.run()
probs = [abs(a)**2 for a in state]
plot_bars(list_to_dict(probs, False), f'v = {v}', 'Outcomes', 'Probabilities')

In [ ]:
result = qc.measure(shots = 100)

result['counts']

#### Section 7.1.2

In [ ]:
result['counts'] = {5: 79, 4: 12, 6: 3, 7: 3, 3: 2, 2: 1}

In [ ]:
from math import sqrt

p_4 = result['counts'][4]/sum(result['counts'].values()) # <1>
p_5 = result['counts'][5]/sum(result['counts'].values())

decimal_estimate = sqrt(p_5)/(sqrt(p_4)+ sqrt(p_5))

In [ ]:
decimal_estimate

## Section 7.2

In [ ]:
def ry_circuit(theta):
    q = QuantumRegister(1)
    qc = QuantumCircuit(q)
    qc.ry(theta, q[0])

    return qc

In [ ]:
def ry_eigen_circuit():
    q = QuantumRegister(1)
    qc = QuantumCircuit(q)

    qc.x(q[0])
    qc.rx(-pi/2, q[0])

    return qc

In [ ]:
from util import print_state_table

qc = ry_eigen_circuit()
state = qc.run()

print_state_table(state)

In [ ]:
from util import all_close

qc = ry_eigen_circuit()
state = qc.run()

assert all_close(state, [1j/sqrt(2), 1/sqrt(2)])

In [ ]:
q = QuantumRegister(1)
qc = QuantumCircuit(q)

qc.x(q[0])
qc.rx(-pi/2, q[0])

theta = pi/6

qc.ry(2*theta, q[0])

state = qc.run()

In [ ]:
print_state_table(state)

In [ ]:
q = QuantumRegister(1)
qc = QuantumCircuit(q)

qc.x(q[0])
qc.rx(-pi/2, q[0])

theta = pi/6

qc.ry(2*theta, q[0]) # <1>
qc.ry(2*theta, q[0]) # <1>

state = qc.run()

In [ ]:
print_state_table(state)

### Section 7.3

In [ ]:
n = 3
q = QuantumRegister(n)
a = QuantumRegister(1)
qc = QuantumCircuit(q, a)

In [ ]:
qc.x(a[0])
qc.rx(-pi/2, a[0])

In [ ]:
for i in range(n):
    qc.h(q[i])

theta = 4.7*2*pi/2**n

for i in range(n):
    for _ in range(2**i):
        qc.cry(2*theta, q[i], a[0])  # <1> 

In [ ]:
qc.rx(pi/2, a[0])
qc.x(a[0])

In [ ]:
qc.iqft(q)

In [ ]:
from util import all_close
from math import cos

state = qc.run()

n = 3
theta = 4.7*2*pi/2**n
s = complex_sincd(n, theta/(2*pi)*2**n)
assert all_close(state[:2**n], s)

### Section 7.4

In [ ]:
def ry_phase_encoding_from_eigenstate(n, theta):
    
    q = QuantumRegister(n)
    a = QuantumRegister(1)
    qc = QuantumCircuit(q, a)

    qc.append(ry_eigen_circuit(), a)
    qc.report('eigenstate')

    for i in range(n):
        qc.h(q[i])

    for i in range(n):
        for _ in range(2**i):
            qc.c_append(ry_circuit(2*theta), q[i], a)
            
    qc.report('geometric_sequence_superposition')
    
    qc.append(ry_eigen_circuit().inverse(), a)

    qc.report('geometric_sequence')

    qc.iqft(q)
    qc.report('estimate')

    return qc

In [ ]:
n = 3
theta = 4.7*2*pi/2**n
qc = ry_phase_encoding_from_eigenstate(n, theta)

In [ ]:
from util_qiskit import hume_to_qiskit

qc_qiskit = hume_to_qiskit(qc.regs, qc.reports['eigenstate'][1])
print(qc_qiskit)

In [ ]:
print_state_table(qc.reports['eigenstate'][2])

In [ ]:
qc_qiskit = hume_to_qiskit(qc.regs, qc.reports['geometric_sequence_superposition'][1])
print(qc_qiskit)

In [ ]:
print_state_table(qc.reports['geometric_sequence_superposition'][2])

In [ ]:
print_state_table(qc.reports['geometric_sequence'][2])

In [ ]:
qc_qiskit = hume_to_qiskit(qc.regs, qc.reports['estimate'][1])
print(qc_qiskit)

In [ ]:
print_state_table(qc.reports['estimate'][2])

### Section 7.5

Listing 7.4

In [ ]:
def phase_estimation_circuit(n, circuit, eigen_circuit=None):
    q = QuantumRegister(n)
    a = QuantumRegister(sum(circuit.regs))
    qc = QuantumCircuit(q, a) # <1>

    if eigen_circuit is not None:
        qc.append(eigen_circuit, a)
        qc.report('eigenstate')

    for i in range(n):
        qc.h(q[i])

    for i in range(n):
        for _ in range(2**i):
            qc.c_append(circuit, q[i], a)

    qc.report('geometric_sequence_superposition')

    if eigen_circuit is not None:
        qc.append(eigen_circuit.inverse(), a)
        qc.report('geometric_sequence')

    qc.iqft(q)
    qc.report('estimate')

    return qc

In [ ]:
n = 3
N = 2**n
theta = 4.7*2*pi/N

qc = phase_estimation_circuit(n, ry_circuit(2*theta), ry_eigen_circuit())

In [ ]:
eig = qc.reports['eigenstate'][2]
assert all_close(eig, [1j/sqrt(2) if k == 0 else 0 for k in range(N)] +
    [1/sqrt(2) if k == 0 else 0 for k in range(N)])

geom = qc.reports['geometric_sequence'][2]
g = [1/sqrt(N)*cis(k*theta) for k in range(N)]
assert all_close(geom[:N], g)

estimate = qc.reports['estimate'][2]
s = complex_sincd(n, theta/(2*pi)*N)
assert all_close(estimate, s + [0 for _ in range(N)])

### Section 7.6

In [ ]:
def phase_estimation_circuit(n, circuit, eigen_circuit=None, swap=True):
    q = QuantumRegister(n)
    a = QuantumRegister(sum(circuit.regs))
    qc = QuantumCircuit(q, a) # a is last (at the bottom)

    if eigen_circuit is not None:
        qc.append(eigen_circuit, a)
    qc.report('eigenstate')

    for i in range(n):
        qc.h(q[i])

    for i in range(n):
        for _ in range(2**i):
            if swap:
                qc.c_append(circuit, q[i], a)
            else:
                qc.c_append(circuit, q[n-1-i], a) # <1>
                
    qc.report('geometric_sequence_superposition')

    if eigen_circuit is not None:
        qc.append(eigen_circuit.inverse(), a)

    qc.report('geometric_sequence')

    qc.iqft(q if swap else q[::-1], swap)
    qc.report('estimate')

    return qc

In [ ]:
n = 3
N = 2**n
theta = 4.7*2*pi/N
swap = False

qc = phase_estimation_circuit(n, ry_circuit(2*theta), ry_eigen_circuit(), swap)

In [ ]:
state = qc.run()
assert all_close(state, s + [0 for _ in range(N)])

In [ ]:
from util_qiskit import show_reports
show_reports(qc)

In [ ]:
from util import reverse_index_state

geom = qc.reports['geometric_sequence'][2]
g = [1/sqrt(N)*cis(k*theta) for k in range(N)]
assert all_close(geom[:N], g if swap else reverse_index_state(g)) # <1>

In [ ]:
estimate = qc.reports['estimate'][2]
s = complex_sincd(n, theta/(2*pi)*N)
assert all_close(estimate, s + [0 for _ in range(N)])

In [ ]:
from random import uniform

def test_ry_phase_estimation():
    n = 3
    N = 2**n

    random_thetas = [uniform(-pi, pi) for _ in range(10)]

    for theta in random_thetas:
        for swap in [True, False]:
            qc = phase_estimation_circuit(n, ry_circuit(theta), ry_eigen_circuit(), swap)

            eig = qc.reports['eigenstate'][2]
            assert all_close(eig, [1j/sqrt(2) if k == 0 else 0 for k in range(N)] +
                             [1/sqrt(2) if k == 0 else 0 for k in range(N)])

            geom = qc.reports['geometric_sequence'][2]
            g = [1/sqrt(N)*cis(k*theta/2) for k in range(N)]
            assert all_close(geom[:N], g if swap else reverse_index_state(g))

            estimate = qc.reports['estimate'][2]
            s = complex_sincd(n, theta/2/(2*pi)*N)
            assert all_close(estimate[:N], s)

In [ ]:
test_ry_phase_estimation()

### Additional tests

In [ ]:
from util_qiskit import same_as_qiskit

for n in range(10):
    for eigen in [True, False]:
        for swap in [True, False]:
            qc = phase_estimation_circuit(n, ry_circuit(theta), ry_eigen_circuit() if eigen else None, swap)
            assert same_as_qiskit(qc)

In [ ]:
from math import sin
from util import rev, inner
from algo import fourier_basis

def test_ry_phase_estimation_no_eigen():
    n = 3
    N = 2**n

    random_thetas = [uniform(-pi, pi) for _ in range(10)]

    for theta in random_thetas:
        for swap in [True, False]:
            qc = phase_estimation_circuit(n, ry_circuit(theta), None, swap)
            geom = qc.reports['geometric_sequence'][2]
            if swap:
                g = [1/sqrt(N)*cos(k*theta/2) for k in range(N)] + [1/sqrt(N)*sin(k*theta/2) for k in range(N)]
            else:
                g = [1/sqrt(N)*cos(rev(n, k)*theta/2) for k in range(N)] + [1/sqrt(N)*sin(rev(n, k)*theta/2) for k in range(N)]

            # print(g)
            assert all_close(geom, g)

            e = qc.reports['estimate'][2]
            # tabulate_state(e)

            g = [1/sqrt(N)*cos(k*theta/2) for k in range(N)] + [1/sqrt(N)*sin(k*theta/2) for k in range(N)]
            f = ([inner(g[:N], fourier_basis(N, k)) for k in range(N)] +
                 [inner(g[N:], fourier_basis(N, k)) for k in range(N)])
            # fourier_basis(N, k)) = [sqrt(1/N) * cis(j*k*2*pi/N) for j in range(N)]
            # print(f)
            assert all_close(e, f)

            v = 2**(n-2)*theta/pi

            c = complex_sincd(n, v)
            assert all_close(c, [e[k] + 1j*e[k+N] for k in range(N)])

            d = complex_sincd(n, N-v)
            assert all_close(e, [(c[k] + d[k])/2 for k in range(N)] + [-1j*(c[k] - d[k])/2 for k in range(N)])

In [ ]:
test_ry_phase_estimation_no_eigen()